In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

In [2]:
def breaker() -> None: print("\n" + 50*"*" + "\n")

In [3]:
class CFG(object):
    def __init__(self,
                 seed: int = 42,
                 n_splits: int = 5,
                 show_info: bool = False,
                 ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.features_path = "../input/kps-trsfs-na384"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

In [4]:
class Pipelines(object):
    def __init__(self, model_name: str, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("classifier", LogisticRegression(random_state=seed, max_iter=1000)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("classifier", KNeighborsClassifier()),
                ]
            )

        elif self.model_name == "svc":
            self.model = Pipeline(
                steps=[
                    ("classifier", SVC(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "xgb":
            self.model = Pipeline(
                steps=[
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )

In [5]:
names: list = [
    "lgr", 
    "knc", 
    "rfc",
    "xgb",
    "svc"
]

model_names: list = [
    "swin_base_patch4_window12_384_in22k",
    "swin_large_patch4_window12_384_in22k",
    "vit_base_patch16_384",
    "vit_base_patch32_384",
    "vit_large_patch16_384",
    "vit_large_patch32_384",
    "deit3_base_patch16_384_in21ft1k",
    "deit3_large_patch16_384_in21ft1k",
    "deit3_small_patch16_384_in21ft1k",
    "deit_base_distilled_patch16_384",
]

overall_best_acc: float = 0.0

y = pd.read_csv("../input/kps-dataframe/dataframe.csv")
y = y.label_index.copy().values

for model_name in model_names:
    X = np.load(os.path.join(cfg.features_path, f"{model_name}.npy"))
    if cfg.show_info: 
        breaker()
        print(f"{model_name.upper()}")
    for name in names:
        fold: int = 1
        best_name: str = None
        best_acc: float = 0.0
        if cfg.show_info: breaker()
        for tr_idx, va_idx in StratifiedKFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X, y):
            X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
            my_pipeline = Pipelines(name, cfg.seed)
            my_pipeline.model.fit(X_train, y_train)

            y_pred = my_pipeline.model.predict(X_valid)
            if cfg.show_info:
                acc = accuracy_score(y_pred, y_valid)
                print(f"{my_pipeline.model_name}, {fold}, Accuracy : {acc:.5f}")
            else:
                acc = accuracy_score(y_pred, y_valid)

            if acc > overall_best_acc:
                overall_best_acc = acc
                overall_best_name = f"{model_name}_{name}_{fold}"
            
            if acc > best_acc:
                best_acc = acc
                best_name = f"{model_name}_{name}_{fold}"

            with open(os.path.join(cfg.model_save_path, f"{model_name}_{name}_{fold}.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
            fold += 1
        
        print(f"\nBest : {best_name}")
    

if cfg.show_info: 
    breaker()
    print(f"Overall Best Model : {overall_best_name}")

breaker()


**************************************************

SWIN_BASE_PATCH4_WINDOW12_384_IN22K

**************************************************



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 1, Accuracy : 0.69215


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 2, Accuracy : 0.69040


...

lgr, 3, Accuracy : 0.59358


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 4, Accuracy : 0.60300


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 5, Accuracy : 0.59043

Best : deit_base_distilled_patch16_384_lgr_4

**************************************************

knc, 1, Accuracy : 0.49878
knc, 2, Accuracy : 0.49494
knc, 3, Accuracy : 0.49127
knc, 4, Accuracy : 0.49860
knc, 5, Accuracy : 0.49791

Best : deit_base_distilled_patch16_384_knc_1

**************************************************

rfc, 1, Accuracy : 0.53543
rfc, 2, Accuracy : 0.54380
rfc, 3, Accuracy : 0.51571
rfc, 4, Accuracy : 0.53806
rfc, 5, Accuracy : 0.53946

Best : deit_base_distilled_patch16_384_rfc_2

**************************************************

xgb, 1, Accuracy : 0.56021
xgb, 2, Accuracy : 0.56161
xgb, 3, Accuracy : 0.55203
xgb, 4, Accuracy : 0.55831
xgb, 5, Accuracy : 0.56913

Best : deit_base_distilled_patch16_384_xgb_5

**************************************************

svc, 1, Accuracy : 0.58290
svc, 2, Accuracy : 0.58185
svc, 3, Accuracy : 0.56425
svc, 4, Accuracy : 0.58310
svc, 5, Accuracy : 0.58624

Best : deit_base_distilled_patch16_3